In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# Normal label
normal_label = "Benign"
# Base path where to find the dataset
base_path = "../data/IDS2018/original"
# Base path where processed dataset will be stored
export_path = "../data/IDS2018"
# Name of the file summarizing the preprocessing
info_fname = "ids2018_info.csv"
# File name of the cleaned/processed dataset
export_fname = "ids2018.csv"
# Used to track preprocessing steps
stats = defaultdict()
# Columns to drop before any analysis
cols_to_drop = [
    'Dst IP',
    'Flow ID',
    'Src IP',
    'Src Port',
    'Protocol',
    'Timestamp',
    'Dst Port'
]

In [2]:
df = pd.DataFrame()
for f in os.listdir(base_path):
    chunk = pd.read_csv(os.path.join(base_path, f))
    chunk.columns = chunk.columns.str.strip()
    chunk.drop(cols_to_drop, axis=1, errors="ignore")
    df = pd.concat((df, chunk))
    print(f)
stats["dropped_cols"] = ""
stats["n_dropped_cols"] = 0
stats["n_dropped_rows"] = 0
stats["n_instances"] = len(df)
stats["n_features"] = df.shape[1] - 1
stats["anomaly_ratio"] = "{:2.4f}".format((df["Label"] != normal_label).sum() / len(df))
print(stats)
df.to_csv(export_path + "/ids2018_merged.csv", index=False)
df.head(5)

/home/local/USHERBROOKE/verj2009/anaconda3/envs/nrcan/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3106: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
defaultdict(None, {'dropped_cols': '', 'n_dropped_cols': 0, 'n_dropped_rows': 0, 'n_instances': 16233002, 'n_features': 83, 'anomaly_ratio': '0.1693'})


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Flow ID,Src IP,Src Port,Dst IP
0,443,6,28/02/2018 08:22:13,94658,6,7,708,3718,387,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN
1,443,6,28/02/2018 08:22:13,206,2,0,0,0,0,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN
2,445,6,28/02/2018 08:22:15,165505,3,1,0,0,0,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN
3,443,6,28/02/2018 08:22:16,102429,6,7,708,3718,387,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN
4,443,6,28/02/2018 08:22:16,167,2,0,0,0,0,0,...,0,0.0,0.0,0,0,Benign,NaN,NaN,NaN,NaN


In [3]:
df = pd.read_csv(export_path + "/ids2018_merged.csv")
df.head(5)

/home/local/USHERBROOKE/verj2009/anaconda3/envs/nrcan/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3106: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,94658,6,7,708,3718,387,0,118.0,159.284651,1460,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,206,2,0,0,0,0,0,0.0,0.0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,165505,3,1,0,0,0,0,0.0,0.0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,102429,6,7,708,3718,387,0,118.0,159.284651,1460,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,167,2,0,0,0,0,0,0.0,0.0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [4]:
df.columns

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
   

## Inner class imbalance
Between anomalies, there is a strong class imbalance

In [4]:
# Original class imbalance within attacks themselves
mask = df["Label"] != normal_label
original_ad_ratios = pd.DataFrame(
    pd.concat(
        (df[mask]["Label"].value_counts(),
        df[mask]["Label"].value_counts() / len(df[mask])), axis=1),
)
original_ad_ratios.to_csv(export_path + "/ids2018_anomaly_labels_ratio.csv")
original_ad_ratios.columns = ["Count", "Ratio"]
original_ad_ratios

,Count,Ratio
DDOS attack-HOIC,686012,0.249614
DDoS attacks-LOIC-HTTP,576191,0.209654
DoS attacks-Hulk,461912,0.168072
Bot,286191,0.104134
FTP-BruteForce,193360,0.070356
SSH-Bruteforce,187589,0.068257
Infilteration,161934,0.058922
DoS attacks-SlowHTTPTest,139890,0.050901
DoS attacks-GoldenEye,41508,0.015103
DoS attacks-Slowloris,10990,0.003999


We regroup similar attacks under the same label

In [5]:
# Group DoS attacks
mask = df["Label"].str.startswith("DoS")
df.loc[mask, "Label"] = "DoS"

# Group DDoS attacks
mask = df["Label"].str.lower().str.startswith("ddos")
df.loc[mask, "Label"] = "DDoS"

# Group Web attacks
mask = df["Label"].str.startswith("Web Attack")
df.loc[mask, "Label"] = "Web Attack"

In [6]:
# Updated class imbalance
mask = df["Label"] != normal_label
mod_ad_ratios = pd.DataFrame(
    pd.concat(
        (df[mask]["Label"].value_counts(),
        df[mask]["Label"].value_counts() / len(df[mask])), axis=1),
)
mod_ad_ratios.columns = ["Count", "Ratio"]
mod_ad_ratios

,Count,Ratio
DDoS,1263933,0.459897
DoS,654300,0.238075
Bot,286191,0.104134
FTP-BruteForce,193360,0.070356
SSH-Bruteforce,187589,0.068257
Infilteration,161934,0.058922
Brute Force -Web,611,0.000222
Brute Force -XSS,230,0.000084
SQL Injection,87,0.000032
Label,59,0.000021


## Check unique values
Drop columns with unique values

In [7]:
uniq_cols = df.columns[df.nunique() <= 1].tolist()
stats["n_unique_cols"] = len(uniq_cols)
if uniq_cols:
    print("Found {} columns with unique values: {}".format(len(uniq_cols), uniq_cols))
    stats["unique_cols"] = ", ".join([str(col) for col in uniq_cols])
    df.drop(uniq_cols, axis=1, inplace=True)
    stats["n_dropped_cols"] += len(uniq_cols)
    uniq_cols = df.columns[df.nunique() <= 1].tolist()
assert len(uniq_cols) == 0, "Found {} columns with unique values: {}".format(len(uniq_cols), uniq_cols)
print("Columns are valid with more than one distinct value")

Columns are valid with more than one distinct value


## Check for NaN/invalid values
First, find the columns with NaN values. Further processing will be required if we find any.

In [8]:
# Replacing INF values with NaN
df = df.replace([-np.inf, np.inf], np.nan)
nan_cols = df.columns[df.isna().sum() > 0].tolist()
stats["n_nan_cols"] = len(nan_cols)
if nan_cols:
    stats["nan_cols"] = ", ".join([str(col) for col in nan_cols])
print("Found NaN columns: {}".format(nan_cols))

Error: Canceled future for execute_request message before replies were done

Having found two columns with NaN values, we must investigate further before taking any decision.

In [ ]:
df[nan_cols].isna().sum()

Are the nan instances associated with anomalies?

In [ ]:
df[df.isna().any(axis=1)]["Label"].value_counts()

Dropping them seems risky since we also drop anomalies which are already scarce

In [ ]:
# Check different values in Flow Duration
print(df[df.isna().any(axis=1)]["Flow Duration"].unique())
# Count number of nan instances when `Flow Duration` > 0
df[df["Flow Duration"] > 0].isna().sum().sum()

`Flow Bytes/s` and `Flow Packets/s` must be computed from `Flow Duration`. When the latter columns is zero, a division by zero occurs and the first two columns have NaN values. Zero values in `Flow Duration` are probably due to a lack of precision in the data type used. They must be associated with flows that lasted nanoseconds. Hence, we can convert the NaN rows to zeros.

In [ ]:
ratio = df[nan_cols].isna().sum()[0] / len(df)
df = df.fillna(0)
print("Replaced {:2.4f}% of original data".format(ratio))
remaining_nans = df.isna().sum().sum()
assert remaining_nans == 0, "There are still {} NaN values".format(remaining_nans)

## Check for negative values
Most of the features should be strictly positive. For instance, a packet with a negative number of bytes makes no sense.

In [ ]:
num_cols = df.select_dtypes(exclude="object").columns
mask = (df[num_cols] < 0).sum() > 0
neg_cols = df[num_cols].columns[mask]
stats["n_negative_cols"] = len(neg_cols)
stats["negative_cols"] = ", ".join(neg_cols)
print("Found {} columns with negative values: {}".format(len(neg_cols), neg_cols))

In [ ]:
neg_df = pd.DataFrame(
    pd.concat((
        (df[neg_cols] < 0).sum(),
        (df[neg_cols] < 0).sum() / len(df)
    ), axis=1)
)
neg_df.columns = ["Count", "Ratio"]
neg_df = neg_df.sort_values("Count", ascending=False)
neg_df

In [ ]:
print((df["Init_Win_bytes_backward"][df["Init_Win_bytes_backward"] < 0]).unique())
print((df["Init_Win_bytes_forward"][df["Init_Win_bytes_forward"] < 0]).unique())
df[df["Init_Win_bytes_backward"] < 0]

In [ ]:
# Drop `Init_Win_bytes_forward` and `Init_Win_bytes_backward` because too many of their values are equal to -1 which makes no sense.
to_drop = neg_df[neg_df["Ratio"] > 0.01].index.tolist()
df = df.drop(to_drop, axis=1)
neg_df = neg_df.drop(to_drop)
stats["n_dropped_cols"] += len(to_drop)
stats["dropped_cols"] = stats["dropped_cols"] + ", ".join(to_drop)
num_cols = df.select_dtypes(include=np.number).columns
print("Dropped {} columns: {}".format(len(to_drop), to_drop))

In [ ]:
df[(df[num_cols] < 0).any(1)]["Label"].value_counts()

In [ ]:
df[(df[num_cols] < 0).any(1)]

When Flow Duration < 0, multiple columns are negative. Since these rows are only associated with BENIGN flows, we can drop them.

In [ ]:
n_dropped = (df["Flow Duration"] < 0).sum()
stats["n_dropped_rows"] += n_dropped
df = df[df["Flow Duration"] >= 0]
print("Dropped {} rows".format(n_dropped))

In [ ]:
neg_cols_when_anomalies = df[num_cols].columns[(df[num_cols][((df[num_cols]).any(1)) & (df["Label"] != normal_label)] < 0).sum() > 0]
neg_cols_when_anomalies

In [ ]:
t = neg_cols_when_anomalies
df[(df[t] < 0).any(1)]

In [ ]:
to_drop = list(neg_cols_when_anomalies)
stats["n_dropped_cols"] += len(neg_cols_when_anomalies)
stats["dropped_cols"] = stats["dropped_cols"] + ", ".join(to_drop)
df = df.drop(to_drop, axis=1)
print("Dropped {} columns {}".format(len(to_drop), to_drop))

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns
neg_cols_labels = df[(df[num_cols] < 0).any(1)]["Label"].unique()
assert len(neg_cols_labels) == 1 and neg_cols_labels[0] == normal_label
idx_to_drop = df[(df[num_cols] < 0).any(1)].index
n_dropped = len(idx_to_drop)
stats["n_dropped_rows"] += n_dropped
df = df.drop(idx_to_drop, axis=0)
print("Dropped {} rows".format(n_dropped))
assert (df[num_cols] < 0).any(1).sum() == 0, "There are still negative values"
print("There are no more negative values")

In [ ]:
df = df.drop(["Destination Port"], axis=1)
df["Category"] = df["Label"]
df.loc[df["Label"] == normal_label, "Label"] = 0
df.loc[df["Label"] != 0, "Label"] = 1

## Normalize attributes

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
assert np.allclose(df[num_cols].max(axis=0).to_numpy(), 1.), "Found values different than 1."
assert np.allclose(df[num_cols].min(axis=0).to_numpy(), 0.), "Found values lesser than 0."
print("Data is scaled between 0 and 1")

In [ ]:
stats["n_final_features"] = df.shape[1] - 2
stats["n_final_rows"] = df.shape[0]
stats["final_anomaly_ratio"] = (df["Label"] != 0).sum() / len(df)

## Store processed dataset to CSV

In [ ]:
df.to_csv(export_path + "/" + export_fname, index=False)
print("Processed data saved under: {}".format(base_path + "/" + export_fname))

## Store basic information

In [ ]:
stats_df = pd.DataFrame(stats, index=[0])
stats_df.to_csv(export_path + "/" + info_fname, index=False)
print("Processing summary saved under: {}".format(base_path + "/" + info_fname))
stats_df